Getting simple account info based on an acc address

In [1]:
from solana.rpc.api import Client
from solders.pubkey import Pubkey

# Define known program IDs
RAYDIUM_CLMM_PROGRAM = "CAMMCzo5YL8w4VFF8KVHrK22GGUsp5VTaW7grrKgrWqK"  # Raydium Concentrated Liquidity Market Maker
RAYDIUM_AMM_PROGRAM = "675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8"  # Raydium Standard AMM (CPMM)
METEORA_DLMM_PROGRAM = "LBUZKhRxPF3XUpBCjp4YzTKgLccjZhTSDM9YuVaPwxo"  # Meteora Dynamic Liquidity Market Maker
ORCA_WHIRLPOOL_PROGRAM = "whirLbMiicVdio4jvUfM5KAg6Ct8VwpasGwh5RV6a"  # Orca Whirlpools (CLMM)
LIFINITY_PROGRAM = "EewxydAPJAb2sBCLQeoVwR1n6yUpG7djL2T3nBdw2A2n"  # Lifinity Proactive Market Maker
PHOENIX_PROGRAM = "PhoeniXxDaP9rssfjbPPbFR7bGCDNE6SR9t6Fc1nxJ"  # Phoenix Order Book DEX
CREMA_CLMM_PROGRAM = "CRMaCEMsB9d6vM6V9D5q5g5vSUr9N2nT3BPCzppMJr9h"  # Crema Finance CLMM

# 1. Connect to Solana
client = Client("https://api.mainnet-beta.solana.com")

# 2. Account info
account = Pubkey.from_string("9VSs58bnJ2KQ2MBeBf19RPH7AMHAysHYtcHEZAXsjNY4")
account_info = client.get_account_info(account)

# 3. Print account info
print(account_info)
print(account)




GetAccountInfoResp { context: RpcResponseContext { slot: 332080474, api_version: Some("2.1.16") }, value: Some(Account(Account { lamports: 11837697340, data.len: 0, owner: 11111111111111111111111111111111, executable: false, rent_epoch: 18446744073709551615 })) }
9VSs58bnJ2KQ2MBeBf19RPH7AMHAysHYtcHEZAXsjNY4


In [2]:
# --- SIMPLE SOLANA TRANSACTION VIEWER ---

# 1. Setup Python path to find the scripts module
import sys
import os
project_root = "/workspaces/DLMM-repo"  # The root of your project
if project_root not in sys.path:
    sys.path.append(project_root)

# 2. Import basic libraries
from scripts.solana_client import get_client  # Your Helius client
from solders.pubkey import Pubkey
import pandas as pd
from datetime import datetime

# 3. Set the Solana account address to analyze
account_address = "9VSs58bnJ2KQ2MBeBf19RPH7AMHAysHYtcHEZAXsjNY4"  # The account you want to check
pubkey = Pubkey.from_string(account_address)  # Convert to Solana Pubkey object

# 4. Get your Solana client (using Helius RPC)
client = get_client()

# 5. Get recent transactions for this account (adjust limit as needed)
print(f"Getting transactions for account: {account_address}")
signatures_response = client.get_signatures_for_address(pubkey, limit=10)

# 6. Create a list to store transaction data
transactions = []

# 7. Process each transaction signature
for sig_info in signatures_response.value:
    # Extract basic information from the signature info
    tx_data = {
        "signature": sig_info.signature,
        "slot": sig_info.slot,
        "timestamp": datetime.fromtimestamp(sig_info.block_time),
        "success": "Yes" if sig_info.err is None else "No"
    }
    transactions.append(tx_data)
    
    # Print basic info as we process
    print(f"Found transaction: {sig_info.signature} at {datetime.fromtimestamp(sig_info.block_time)}")

# 8. Create a DataFrame for better visualization
df = pd.DataFrame(transactions)

# 9. Display the results in a nice table
print("\n--- TRANSACTION HISTORY ---")
display(df)

# 10. Print some basic statistics
print(f"\nTotal transactions found: {len(transactions)}")
print(f"Successful transactions: {len([t for t in transactions if t['success'] == 'Yes'])}")
print(f"Failed transactions: {len([t for t in transactions if t['success'] == 'No'])}")

ModuleNotFoundError: No module named 'scripts'

Retrieving Info about LP pair

In [15]:
# --- RAYDIUM LIQUIDITY PAIR EXPLORER ---

# 1. Setup Python path
import sys
import os
project_root = "/workspaces/DLMM-repo"
if project_root not in sys.path:
    sys.path.append(project_root)

# 2. Import libraries
from scripts.solana_client import get_client
from solders.pubkey import Pubkey
import base64
import struct

# 3. Get your Solana client
client = get_client()

# 4. Define the liquidity pair address
# Example: RAY-USDC pair on Raydium
# Change this to the pool you want to examine
pool_address = "GFN5HKGs9Zxk9rmDWToWiBEE57e7ELK8EFd8JQ9iaPzy"

print(f"Exploring liquidity pair: {pool_address}")

# 5. Get account info
try:
    account_info = client.get_account_info(Pubkey.from_string(pool_address))
    
    if not account_info.value:
        print(f"No account found at address: {pool_address}")
    else:
        # 6. Print basic account info
        print(f"\nAccount Owner: {account_info.value.owner}")
        print(f"Data Length: {len(account_info.value.data)} bytes")
        
        # 7. Different DEXs have different data layouts
        # Raydium's AMM pools have specific data structure
        # This is a simplified approach - actual offsets depend on the program version
        
        # For educational purposes, let's see the first part of the data
        data_bytes = account_info.value.data
        print(f"\nFirst 32 bytes of data: {data_bytes[:32].hex()}")
        
        # 8. For Raydium pools, the LP token mint is at a specific offset
        # Note: This may vary by program version and pool type
        # Raydium v4 pools typically have the LP mint at offset 200
        if len(data_bytes) >= 232:
            # Try common offsets for LP token mint
            # Different Raydium versions have different layouts
            possible_offsets = [200, 168, 232, 104]
            
            print("\nPossible LP token addresses (examine these to find the correct one):")
            for offset in possible_offsets:
                if offset + 32 <= len(data_bytes):
                    try:
                        mint_bytes = data_bytes[offset:offset+32]
                        mint_address = str(Pubkey.from_bytes(mint_bytes))
                        print(f"  Offset {offset}: {mint_address}")
                        
                        # Check if it's a valid mint account
                        mint_info = client.get_account_info(Pubkey.from_string(mint_address))
                        if mint_info.value:
                            owner = str(mint_info.value.owner)
                            if owner == "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA":  # Token Program
                                print(f"  ✓ This appears to be a valid token mint (owned by Token Program)")
                            else:
                                print(f"  ✗ Not a token mint account (owned by {owner})")
                        else:
                            print(f"  ✗ Account does not exist")
                    except Exception as e:
                        print(f"  ✗ Error parsing offset {offset}: {str(e)}")
        else:
            print("Account data is too short to contain a LP token mint")
            
        # 9. For educational purposes, show what other data might be in the pool
        print("\nMost pools also contain information about:")
        print("- The two tokens in the pair")
        print("- Current reserves/balances for each token")
        print("- Fees and price information")
        print("- Authority addresses")
        
        print("\nTip: To get exact fields for your specific DEX, check the program's documentation or GitHub")

except Exception as e:
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

print("\n--- LEARNING RESOURCES ---")
print("1. Raydium docs: https://raydium.gitbook.io/raydium/")
print("2. Solana dev docs: https://docs.solana.com/developing/clients/javascript-api")
print("3. Solana Cookbook: https://solanacookbook.com/")


Exploring liquidity pair: GFN5HKGs9Zxk9rmDWToWiBEE57e7ELK8EFd8JQ9iaPzy

Account Owner: CAMMCzo5YL8w4VFF8KVHrK22GGUsp5VTaW7grrKgrWqK
Data Length: 1544 bytes

First 32 bytes of data: f7ede3f5d7c3de46fd57670bc5297b9ae5c50a95679dff0df4b2498452b693b9

Possible LP token addresses (examine these to find the correct one):
  Offset 200: D4G8AMkApk7EADuzw7RTHwmMoWgxshx7QyLecKXk3fxe
  ✗ Account does not exist
  Offset 168: EwcHM8WYBoEQYuGo3cNStXze9A5X32dy4EidX1KqP3TU
  ✗ Account does not exist
  Offset 232: 9DNiprCXrEYDHijH4ZyP1CNf6jtabPDcrzau8ryCVa7q
  ✗ Account does not exist
  Offset 104: 6iyQJUGkg1tAr2147a4bXU2iSCgnahUfWMy2vNz86rr
  ✗ Account does not exist

Most pools also contain information about:
- The two tokens in the pair
- Current reserves/balances for each token
- Fees and price information
- Authority addresses

Tip: To get exact fields for your specific DEX, check the program's documentation or GitHub

--- LEARNING RESOURCES ---
1. Raydium docs: https://raydium.gitbook.io/raydium/


Function to Determine what type of Pool we have

In [3]:
from solana.rpc.api import Client
from solders.pubkey import Pubkey

def identify_pool_type(pool_address):
    """
    Determines the type of Solana liquidity pool based on its program ID
    
    Args:
        pool_address: A Solders Pubkey object representing the pool address
        
    Returns:
        A dictionary with information about the pool type, program ID, and DEX
    """
    try:
        # Get account info to determine program ID
        solana_client = Client("https://api.mainnet-beta.solana.com")
        account_info = solana_client.get_account_info(pool_address)
        
        if not account_info.value:
            return {"error": "Pool account not found"}
        
        program_id = str(account_info.value.owner)
        
        # Define known program IDs
        RAYDIUM_CLMM_PROGRAM = "CAMMCzo5YL8w4VFF8KVHrK22GGUsp5VTaW7grrKgrWqK"  # Raydium Concentrated Liquidity Market Maker
        RAYDIUM_AMM_PROGRAM = "675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8"  # Raydium Standard AMM (CPMM)
        METEORA_DLMM_PROGRAM = "LBUZKhRxPF3XUpBCjp4YzTKgLccjZhTSDM9YuVaPwxo"  # Meteora Dynamic Liquidity Market Maker
        ORCA_WHIRLPOOL_PROGRAM = "whirLbMiicVdio4jvUfM5KAg6Ct8VwpasGwh5RV6a"  # Orca Whirlpools (CLMM)
        
        # Determine pool type
        if program_id == RAYDIUM_CLMM_PROGRAM:
            return {
                "pool_type": "CLMM",
                "program_id": program_id,
                "dex": "Raydium"
            }
        elif program_id == RAYDIUM_AMM_PROGRAM:
            return {
                "pool_type": "AMM",
                "program_id": program_id,
                "dex": "Raydium"
            }
        elif program_id == METEORA_DLMM_PROGRAM:
            return {
                "pool_type": "DLMM",
                "program_id": program_id,
                "dex": "Meteora"
            }
        elif program_id == ORCA_WHIRLPOOL_PROGRAM:
            return {
                "pool_type": "CLMM",
                "program_id": program_id,
                "dex": "Orca"
            }
        else:
            return {
                "pool_type": "Unknown",
                "program_id": program_id,
                "message": "Unrecognized pool program"
            }
    
    except Exception as e:
        return {"error": f"Failed to identify pool type: {str(e)}"}


# Example usage
if __name__ == "__main__":
    # Example pool address
    pool_address = 'C6tRBKe69kEXVZHebne3GFypxrRRgVHJTJAoNmiacujd'
    pubkey = Pubkey.from_string(pool_address)
    result = identify_pool_type(pubkey)
    print(result) 

{'pool_type': 'CLMM', 'program_id': 'CAMMCzo5YL8w4VFF8KVHrK22GGUsp5VTaW7grrKgrWqK', 'dex': 'Raydium'}
